This code is a live Speech-To-Text (STT) model utilizing OpenAI Whisper model. It can accurately transcribe live audio from chosen audio inputs into text printed on the shell with little delay (<1s).

Improvements to be done:
- Combine old AudioSegments with new input so no wrong transcription
- Allow user to toggle start / stop using buttons or keyboard inputs
- Allow user to select audio input directly from program
- increase performance of live STT model


In [1]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!apt-get install portaudio19-dev python-all-dev
!pip -q install pydub
!pip install anvil-uplink

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-inki74i1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-inki74i1
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://p

In [8]:
# all imports
from IPython.display import Javascript
from google.colab import output, files
from base64 import b64decode
from pydub import AudioSegment
from io import BytesIO
import whisper
import multiprocessing
import time
import threading
import queue
import anvil.server

# Javascript for prompting local mic access
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
#Function to ask for microphone access
def mic():
    display(Javascript(RECORD))

# Function to listen for speech from user
def listen(sec):
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

# Function to save audio from listen to wav file
def download(audio):
  audio_file = "recorded_audio.wav"
  audio.export(audio_file, format="wav")
  #files.download(audio_file)

#Function to transcribe audio
def translate():
  result = model.transcribe("/content/recorded_audio.wav", language = "en")
  print("you said" + result["text"])
  return result["text"]

#Function to run the STT Model
@anvil.server.callable
def STT():
    mic()
    download(listen(3))
    return translate()


# Load whisper model, types: tiny, base, small, medium, large, large-v2
model = whisper.load_model("small")

#Connect to anvil server
anvil.server.connect("server_2GGMSIC55EQPG6SUEFANETJL-LJWM672NOLIA5LJF")

anvil.server.wait_forever()



<IPython.core.display.Javascript object>

you said


KeyboardInterrupt: 

In [ ]:
# all imports
from IPython.display import Javascript
from google.colab import output, files
from base64 import b64decode
from pydub import AudioSegment
from io import BytesIO
import whisper
import time
import threading
import queue

# Javascript for prompting local mic access
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

# Load whisper model, types: tiny, base, small, medium, large, large-v2
model = whisper.load_model("small")

#Function to ask for microphone access
def mic():
    display(Javascript(RECORD))

# Function to listen for speech from user
def listen(sec):
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

# Function to save audio from listen to wav file
def download(audio):
  audio_file = "recorded_audio.wav"
  audio.export(audio_file, format="wav")
  #files.download(audio_file)

#Transcribe audio
def translate():
  result = model.transcribe("/content/recorded_audio.wav", language = "en")
  print(result["text"])

# Generator to save user speech into queue
def producer(speechQueue, speechLock):
    for i in range(3):

      print("speak now")
      speech = listen(3)

      with speechLock:
        speechQueue.put(speech)
      print("speech added to speechQueue")

#middleman
def communicator(speechQueue, speechLock, event):
    while True:
      if not speechQueue.empty():

        with speechLock:
          speech = speechQueue.get()
        print("Speech has been consumed")

        download(speech)
        event.set()
        print("speech added to translation queue")

# Translates audio segments from translation queue
def translator(event):
    while True:

        print("speech is being translated")
        event.wait()
        translate()

        event.clear()
        print("speech has been translated")

# Create locks to ensure no deadlock
speechLock = threading.Lock()

# Create an event
event = threading.Event()

# Create queues for communication between threads
speechQueue = queue.Queue()

# To access com mic
mic()

# Create and start producer thread
producer_thread = threading.Thread(target=producer, args=(speechQueue, speechLock,), daemon=False)
producer_thread.start()

# Create and start consumer thread
communicator_thread = threading.Thread(target=communicator, args=(speechQueue, speechLock, event,), daemon=False)
communicator_thread.start()

#Create and start a translator thread
translator_thread = threading.Thread(target=translator, args=(event,), daemon=False)
translator_thread.start()

# Wait for both threads to complete
producer_thread.join()
communicator_thread.join()
translator_thread.join()




<IPython.core.display.Javascript object>

speak now
speech is being translated


KeyboardInterrupt: 

In [ ]:
# all imports
from IPython.display import Javascript
from google.colab import output, files
from base64 import b64decode
from pydub import AudioSegment
from io import BytesIO
import whisper
import multiprocessing
import time
import threading
import queue


# Load whisper model, types: tiny, base, small, medium, large, large-v2
model = whisper.load_model("base")
from pydub import AudioSegment

# Load a non-empty audio segment (example: a WAV file)
audio = AudioSegment.from_file("/content/recorded_audio.wav", format="wav")

# Function to save audio from listen to wav file and transcribe
for i in range(2):
  audio += audio
audio_file = "/content/recorded_audio.wav"
audio.export(audio_file, format="wav")
result = model.transcribe("/content/recorded_audio.wav")
files.download(audio_file)
return(result["text"])



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SyntaxError: 'return' outside function (<ipython-input-12-464336b9d192>, line 28)

In [ ]:

#All imports
import threading
import time
import queue
def mic():
    display(Javascript(RECORD))

# Function to listen for speech from user
def listen(sec):
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

# Function to save audio from listen to wav file and transcribe
def translate(audio):
  audio_file = "recorded_audio.wav"
  audio.export(audio_file, format="wav")
  result = model.transcribe("/content/recorded_audio.wav")
  #files.download(audio_file)
  return(result["text"])


# Generator to save user speech into queue
def producer(speechQueue, lock):
    """Generate data and put it into the queue."""
    for i in range(2):
      print("speak now")
      speech = listen(3)
      with lock:
        speechQueue.put(speech)
      print("speech added to speechQueue")

# Retriever for consuming speech from queue
def consumer(speechQueue, lock):
    """Retrieve data from the queue and process it."""
    while True:
      if not speechQueue.empty():
        print("Speech being translated")
        with lock:
          speech = speechQueue.get()
        final = translate(speech)
        print(final)

# Create a Queue for communication between threads
lock = threading.Lock()
speechQueue = queue.Queue()
mic()

# Create and start producer thread, daemon so user can exit whenever they want
producer_thread = threading.Thread(target=producer, args=(speechQueue, lock,), daemon=False)
producer_thread.start()

# Create and start consumer thread, daemon so user can exit whenever they want
consumer_thread = threading.Thread(target=consumer, args=(speechQueue, lock,), daemon=False)
consumer_thread.start()

# Wait for both threads to complete
producer_thread.join()
consumer_thread.join()


